In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [2]:
%reset
import matplotlib
import numpy as np
import scipy as sp
import netCDF4 as nc
import cmocean as cm
from scipy.io import loadmat
import xarray as xr
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
%matplotlib inline

from salishsea_tools import visualisations as vis

ImportError: cannot import name 'is_scalar'

### Plot the 2018 freshet, as it's the freshet in the past 4 years 

In [ ]:
path_to_file = '/results2/SalishSea/nowcast-green.201905/20may18'
fname = 'SalishSea_1h_20180520_20180520_grid_T.nc'

In [ ]:
variable_to_plot = 'vosaline'
hour_of_day_to_plot = 12
x_size_of_plot = 11
y_size_of_plot = 4
vmin = 20
vmax = 32
stepsize = 0.2
do_I_want_to_save_fig = False
figstr = '2018may20_thalweg.png'
tit = '2018 Freshet'
cbar_label = 'Salinity'
titlefontsize = 20
cbarfontsize = 18
cbartickfontsize = 18
axistickfontsize = 18
#colourmap I've chosen, from https://matplotlib.org/cmocean/
t_cmap = cm.cm.dens

In [ ]:
#bathymetry and meshmap
bathy = nc.Dataset('/data/tjarniko/MEOPAR/grid/bathymetry_201702.nc')
mesh = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')

nc_to_plot = nc.Dataset(path_to_file + fname)
data_to_plot = nc_to_plot[variable_to_plot][hour_of_day_to_plot-1,:,:,:]


fig, ax1 = plt.subplots(1,1,figsize=(x_size_of_plot,y_size_of_plot))
cbar = vis.contour_thalweg(ax1, data_to_plot, bathy, mesh, np.arange(vmin, vmax, stepsize), cmap = t_cmap)
ax1.set_title(tit , fontsize = titlefontsize)
cbar.set_label(cbar_label, fontsize = cbarfontsize)
cbar.ax.tick_params(labelsize=cbartickfontsize) 
ax1.tick_params(axis='both', which='major', labelsize=axistickfontsize)


fig.tight_layout()
plt.show()
if do_I_want_to_save_fig == True:
    fig.savefig(figstr)
